# Parallel Computing

## 19.0 Semaphore

[semaphore는 threading 패키지에 이미 존재](https://docs.python.org/3/library/threading.html#semaphore-objects)하지만, 시험 문제로 나올만한 것이죠. 

- 문제: semaphore를 구현하라.
- 솔루션: condition + counter variable 을 사용하면 됩니다. counter variable 이 max 내에 있을 경우에는 사용하도록 해주고, 아니면 wait/notify를 해주면 되죠. 명심할 것은 condition은 항상 이 이디엄으로 사용해야 한다는 것입니다.

```python
# in a thread trying to acquire, check this condition
with cond:
    while condition_not_met:
        cond.wait()
```

```python
# in another thread holding the lock, when finishing the work
counter -= 1
with cond:
    cond.notify()
```

In [1]:
import threading


class Semaphore:
    def __init__(self, max_concurrent):
        self.max_concurrent = max_concurrent
        self.cond = threading.Condition()
        self.counter = 0
        
    def acquire(self):
        with self.cond:
            while self.counter == self.max_concurrent:
                self.cond.wait()
            self.counter += 1
            
    def release(self):
        with self.cond:
            self.counter -= 1
            self.cond.notify()
            
    def __enter__(self):
        self.acquire()
        
    def __exit__(self, *exc):
        self.release()
        
## User
sem = Semaphore(5)

with sem:
    print('random work')

random work


## 19.3 Implement synchronization for two interleaving threads

- 문제: thread 두개로 1~100까지 출력하고 싶다. t1 은 홀수만, t2 는 짝수만 출력하게 만들 때, 순서대로 출력할 수 있을까?
- 솔루션: condition을 잘 쓰는 것이 관건. t1은 condition이 홀수일 때만 사용하고, t2는 condition이 짝수일 때만 사용하도록 만든다. 역시나 `while-idiom`을 잘 쓰면 된다.

```python
while need_to_print_more_numbers:
    with cond:
        while is_not_odd:
            cond.wait()
    print('odd number')
    with cond:
        cond.notify()
```

In [2]:
from threading import Thread, Condition


class MyCondition(Condition):
    def __init__(self):
        super().__init__()
        self.odd = True
        
    def wait_odd(self):
        with self:
            while not self.odd:
                self.wait()
            
    def wait_even(self):
        with self:
            while self.odd:
                self.wait()            
            
    def toggle_turn(self):
        with self:
            self.odd ^= True
            self.notify()

cond = MyCondition()
        
def print_odd():
    for i in range(1, 11, 2):
        cond.wait_odd()
        print(i)
        cond.toggle_turn()
        
def print_even():
    for i in range(2, 11, 2):
        cond.wait_even()
        print(i)
        cond.toggle_turn()
        
t1 = Thread(target=print_odd)
t2 = Thread(target=print_even)
t1.start()
t2.start()

1
2
3
4
5
6
7
8
9


## 19.6 The readers-writers problem

- 문제: reader, writer 가 있다.
    - reader는 읽는 애들만 있으며 언제나 동시에 사용이 가능하다.
    - reader는 writer가 사용하려고 하면 기다려야 한다.
    - writer는 자기가 독점적으로 사용할때까지 기다려야 한다. (즉, 다른 모든 reader, writer 가 일을 멈추어야 시작)
- 솔루션: condition 을 한개 쓰는 버전, 두개 쓰는 버전이 있다.
    - version 1: readers, writers_pending, writers 카운터를 써서 관리
    - version 2: lock_read, lock_write 락들과 readers 카운터를 써서 관리

In [3]:
import threading

class ReadWriteLockV1(threading.Condition):
    def __init__(self):
        self.readers = 0
        self.writers = 0
        self.writers_pending = 0
    
    def read_acquire(self):
        with self:
            while self.writers > 0 or self.writers_pending > 0:
                self.wait()
            self.readers += 1
    
    def read_release(self):
        with self:
            self.readers -= 1
            self.notify()
    
    def write_acquire(self):
        with self:
            self.writers_pending += 1
            while self.writers > 0 or self.readers > 0:
                self.wait()
            self.writers_pending -= 1
            self.writers += 1
    
    def write_release(self):
        with self:
            self.writers -= 1
            self.notify()

10


In [4]:
import threading

class ReadWriteLockV2:
    def __init__(self):
        self.readers = 0
        self.lock_read = threading.Condition()
        self.lock_write = threading.RLock()
    
    def read_acquire(self):
        with self.lock_write:
            pass
        with self.lock_read:
            self.readers += 1
    
    def read_release(self):
        with self.lock_read:
            self.readers -= 1
            self.lock_read.notify()
    
    def write_acquire(self):
        self.lock_write.acquire()
        with self.lock_read:
            while self.readers > 0:
                self.lock_read.wait()
            self.readers += 1
    
    def write_release(self):
        with self.lock_read:
            self.readers -= 1
            self.lock_read.notify()
        self.lock_write.release()